<a href="https://colab.research.google.com/github/GodoyLucas/king_of_rebirth/blob/main/King_of_Rebirth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de desempenho dos players em função do input no campeonado de Call of Duty Warzone King of Rebirth

# Importanto bibliotecas e dados

In [278]:
import pandas as pd
import numpy as np

In [279]:
final = pd.read_csv('https://raw.githubusercontent.com/GodoyLucas/king_of_rebirth/main/king.csv', sep = ";")
final.head()

,player,input,partida,kills,colocacao
0,HARDTREVS,controle,1,5.0,36.0
1,HARDTREVS,controle,2,5.0,12.0
2,HARDTREVS,controle,3,11.0,1.0
3,HARDTREVS,controle,4,3.0,5.0
4,HARDTREVS,controle,5,8.0,7.0


# Verificando condições gerais dos dados

In [280]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   player     352 non-null    object 
 1   input      352 non-null    object 
 2   partida    352 non-null    int64  
 3   kills      348 non-null    float64
 4   colocacao  348 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 13.9+ KB


É possível verificar que, apesar de 4 registros nulos em 'kills' e 'colocacao', as 8 partidas dos 44 players estão presentes no conjunto de dados. 

Os dados nulos são então substituídos por "0" e o tipo dados dos campos são convertidos para inteiro.

In [281]:
final.fillna(value = {'kills': 0, 'colocacao': 0}, inplace = True)
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   player     352 non-null    object 
 1   input      352 non-null    object 
 2   partida    352 non-null    int64  
 3   kills      352 non-null    float64
 4   colocacao  352 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 13.9+ KB


In [282]:
final = final.astype({'kills': int, 'colocacao': int})
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   player     352 non-null    object
 1   input      352 non-null    object
 2   partida    352 non-null    int64 
 3   kills      352 non-null    int64 
 4   colocacao  352 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 13.9+ KB


# Premissas de análise

- A final do campeonato foi realizada em 8 partidas;
- Das 8 partidas, apenas as 5 melhores foram usadas para a classificação geral do campeonato;
- O estudo contempla todas as partidas;
- O campeonato ocorreu no modo solo, ou seja, 1 pessoa por time, competição individual;
- 44 players participaram. Portanto, as colocações nas partidas vão de 1 a 44, não sendo possível empate;
- Nem todos os players participaram e/ou reportaram os resultados de todas as partidas. Houve caso do player não querer jogar, caso de ser desconectado da partida por motivo adverso e caso de não querer reportar o resultado de uma determinada partida. Os resultados faltantes decorrentes desses casos, resultados nulos ou resultados impossíveis dado o contexto serão estimados com base nos resultados do player nas demais partidas.
- Para que a lógica da planilha utilizada para consolidação dos resultados no dia do campeonato não quebrasse, resultados não reportados foram registrados com colocação 44. Este estudo procura identificar os registros legítimos para tratar os que assim só tenham sido reportados pelo uso da planilha.
  - Regristros com colocação 44 com zero kills só são considerados legítimos caso não haja outro registro nesta mesma colocação na partida;
  - Regristros com colocação 44 com mais de zero kills só são considerados legítimos caso não haja outro registro nesta mesma colocação com kills registradas na mesma partida;

# Verificando partidas

A verificação que se segue responderá as seguintes questões:

- Das 44 posições possíveis em cada partida, quais não foram ocupadas?
- Há mais de um player com a mesma colocação? Em quais colocações isso acontece?
- Há registros de colocação preenchidos com '0'?
- Quais players terão seus resultados reavaliados para o estudo?

In [283]:
final.query('colocacao == 44', engine='python').groupby(by='partida')[['colocacao']].count()

,colocacao
partida,
1,8
2,6
3,6
4,3
5,2
6,3
7,10
8,7


Registros com colocação 44 e kills zeradas serão considerados não legítimos, ainda que seja possível ficar na ultima posição sem ter feito nenhuma kill. No entanto, é impossível distinguir a legitimidade de tal registro. Portanto, tais registros serão estimados.

In [284]:
final.query('colocacao == 44 & kills == 0', engine='python').groupby(by='partida')[['colocacao']].count()

,colocacao
partida,
1,8
2,5
3,5
4,2
5,2
6,3
7,9
8,5


Registros de partidas com mais de um player na posição 44 com kills registradas também serão separadas para tratamento, pois não é possível saber qual dos dois registros é legítimo.

In [285]:
final.query('colocacao == 44 & kills != 0', engine='python').groupby(by='partida')[['colocacao']].count()

,colocacao
partida,
2,1
3,1
4,1
7,1
8,2


In [286]:
final.query('(colocacao == 44 | colocacao == 0) & kills == 0', engine='python').groupby(by='partida')[['colocacao']].count()

,colocacao
partida,
1,8
2,5
3,6
4,4
5,4
6,4
7,10
8,10


In [287]:
final.query('colocacao == 44 | colocacao == 0 | colocacao.isnull()', engine='python').groupby(by='partida')[['colocacao']].count()

,colocacao
partida,
1,8
2,6
3,7
4,5
5,4
6,4
7,11
8,12


### Partida 1

In [288]:
partida_1 = final.query('partida == 1').sort_values(by='colocacao', ascending = False)
partida_1.head()

,player,input,partida,kills,colocacao
184,YKZHARDCORE,mouse,1,0,44
72,VOORTZ,mouse,1,0,44
216,NICE PADRIN,controle,1,0,44
136,ILUDIDO_FPS,controle,1,0,44
232,IGOTEE,mouse,1,0,44


Players na 44 colocação:

In [289]:
partida_1.query('colocacao == 44')

,player,input,partida,kills,colocacao
184,YKZHARDCORE,mouse,1,0,44
72,VOORTZ,mouse,1,0,44
216,NICE PADRIN,controle,1,0,44
136,ILUDIDO_FPS,controle,1,0,44
232,IGOTEE,mouse,1,0,44
120,FELPIIS32,mouse,1,0,44
56,RIVOTRIL,mouse,1,0,44
8,VIEIRA,controle,1,0,44


- Não é possível descobrir qual player realmente ficou na 44 colocação nessa partida;
____________________

Colocações não ocupadas:

In [290]:
colocacoes_partida_1 = set(list(partida_1.colocacao))
colocacoes_possiveis = set(list(range(1, 45)))
colocacoes_faltantes_1 = list(colocacoes_possiveis - colocacoes_partida_1)
print(colocacoes_faltantes_1)
colocacoes_faltantes_1.append(44)
colocacoes_faltantes_1

[35, 37, 39, 41, 42, 43, 28]


[35, 37, 39, 41, 42, 43, 28, 44]

- Para além dessas colocações, um player também precisará ser posicionado na 44 colocação;
____________

Colocações com mais de um player:

In [291]:
partida_1.groupby(by = "colocacao")[['player']].count().query('player != 1')

,player
colocacao,
44,8


- Como não foi possível distinguir qual dos registro é o legítimo, todos os 8 players serão reposicionados;
____________________

Colocações '0':

In [292]:
partida_1.query('colocacao == 0')

,player,input,partida,kills,colocacao


- Não houve registro de colocação '0';
___

Players cujos registros serão tratados nessa partida:

In [293]:
players_a_serem_tratados_partida_1 = partida_1.query('colocacao == 44')
players_a_serem_tratados_partida_1

,player,input,partida,kills,colocacao
184,YKZHARDCORE,mouse,1,0,44
72,VOORTZ,mouse,1,0,44
216,NICE PADRIN,controle,1,0,44
136,ILUDIDO_FPS,controle,1,0,44
232,IGOTEE,mouse,1,0,44
120,FELPIIS32,mouse,1,0,44
56,RIVOTRIL,mouse,1,0,44
8,VIEIRA,controle,1,0,44


### Partida 2

In [294]:
partida_2 = final.query('partida == 2').sort_values(by='colocacao', ascending = False)
partida_2.head()

,player,input,partida,kills,colocacao
121,FELPIIS32,mouse,2,0,44
217,NICE PADRIN,controle,2,0,44
57,RIVOTRIL,mouse,2,0,44
329,SAVYTZ,controle,2,0,44
73,VOORTZ,mouse,2,2,44


Players na 44 colocação:

In [295]:
partida_2.query('colocacao == 44')

,player,input,partida,kills,colocacao
121,FELPIIS32,mouse,2,0,44
217,NICE PADRIN,controle,2,0,44
57,RIVOTRIL,mouse,2,0,44
329,SAVYTZ,controle,2,0,44
73,VOORTZ,mouse,2,2,44
9,VIEIRA,controle,2,0,44


- O registro do player VOORTZ é considerado o legítimo e os demais separados para tratamento;
____________________

Colocações não ocupadas:

In [296]:
colocacoes_partida_2 = set(list(partida_2.colocacao))
colocacoes_possiveis = set(list(range(1, 45)))
colocacoes_faltantes_2 = list(colocacoes_possiveis - colocacoes_partida_2)
colocacoes_faltantes_2

[34, 35, 38, 42, 43]

- Colocações a serem preenchidas com tratamento;
____________

Colocações com mais de um player:

In [297]:
partida_2.groupby(by = "colocacao")[['player']].count().query('player != 1')

,player
colocacao,
44,6


- Concluído que só houveram registros equivocados para a posição 44;
____________________

Colocações '0':

In [298]:
partida_2.query('colocacao == 0')

,player,input,partida,kills,colocacao


- Não houve registro de colocação '0';
___

Players cujos registros serão tratados nessa partida:

In [299]:
players_a_serem_tratados_partida_2 = partida_2.query('colocacao == 44 & kills == 0')
players_a_serem_tratados_partida_2

,player,input,partida,kills,colocacao
121,FELPIIS32,mouse,2,0,44
217,NICE PADRIN,controle,2,0,44
57,RIVOTRIL,mouse,2,0,44
329,SAVYTZ,controle,2,0,44
9,VIEIRA,controle,2,0,44


### Partida 3

In [300]:
partida_3 = final.query('partida == 3').sort_values(by='colocacao', ascending = False)
partida_3.head()

,player,input,partida,kills,colocacao
90,KID AIMBOT,controle,3,0,44
338,ANXIETY,mouse,3,0,44
186,YKZHARDCORE,mouse,3,1,44
258,SROGATA,controle,3,0,44
58,RIVOTRIL,mouse,3,0,44


Players na 44 colocação:

In [301]:
partida_3.query('colocacao == 44')

,player,input,partida,kills,colocacao
90,KID AIMBOT,controle,3,0,44
338,ANXIETY,mouse,3,0,44
186,YKZHARDCORE,mouse,3,1,44
258,SROGATA,controle,3,0,44
58,RIVOTRIL,mouse,3,0,44
74,VOORTZ,mouse,3,0,44


- O registro do player YKZHARDCORE é considerado o legítimo e os demais separados para tratamento;
____________________

Colocações '0':

In [302]:
partida_3.query('colocacao == 0')

,player,input,partida,kills,colocacao
18,iErickziN,controle,3,0,0


- O player iErickziN deverá ser reposicionado nessa partida;
___

Colocações com mais de um player:

In [303]:
partida_3.groupby(by = "colocacao")[['player']].count().query('player != 1')

,player
colocacao,
28,2
34,2
44,6


- Concluído que só houveram registros equivocados para as posições 28, 34 e 44;
____________________

Colocações não ocupadas:

In [304]:
colocacoes_partida_3 = set(list(partida_3.colocacao))
colocacoes_possiveis = set(list(range(1, 45)))
colocacoes_faltantes_3 = list(colocacoes_possiveis - colocacoes_partida_3)
print(colocacoes_faltantes_3)
colocacoes_faltantes_3.append(28)
colocacoes_faltantes_3.append(34)
colocacoes_faltantes_3

[35, 36, 39, 41, 42, 43, 24, 31]


[35, 36, 39, 41, 42, 43, 24, 31, 28, 34]

- Colocações a serem preenchidas com tratamento;
____________

Players cujos registros serão tratados nessa partida:

In [305]:
players_a_serem_tratados_partida_3 = partida_3.query('(colocacao == 44 & kills == 0) | colocacao == 0 | colocacao == 28 | colocacao == 34')
players_a_serem_tratados_partida_3

,player,input,partida,kills,colocacao
90,KID AIMBOT,controle,3,0,44
338,ANXIETY,mouse,3,0,44
258,SROGATA,controle,3,0,44
58,RIVOTRIL,mouse,3,0,44
74,VOORTZ,mouse,3,0,44
330,SAVYTZ,controle,3,1,34
202,ABOJEK,controle,3,1,34
98,DETONAMACHO,controle,3,0,28
34,SURILOCAO,mouse,3,0,28
18,iErickziN,controle,3,0,0


### Partida 4

In [306]:
partida_4 = final.query('partida == 4').sort_values(by='colocacao', ascending = False)
partida_4.head()

,player,input,partida,kills,colocacao
67,EUCHAVS,controle,4,0,44
59,RIVOTRIL,mouse,4,0,44
331,SAVYTZ,controle,4,1,44
131,PRUKKZ,controle,4,1,42
27,MAIKI,controle,4,0,40


Players na 44 colocação:

In [312]:
partida_4.query('colocacao == 44')

,player,input,partida,kills,colocacao
67,EUCHAVS,controle,4,0,44
59,RIVOTRIL,mouse,4,0,44
331,SAVYTZ,controle,4,1,44


- O registro do player SAVYTZ é considerado o legítimo e os demais separados para tratamento;
____________________

Colocações '0':

In [313]:
partida_4.query('colocacao == 0')

,player,input,partida,kills,colocacao
83,MAUMAU,controle,4,0,0
179,CLARA FIERCE,controle,4,0,0


- Os players MAUMAU e CLARA FIERCE deverão ser reposicionados nessa partida;
___

Colocações com mais de um player:

In [314]:
partida_4.groupby(by = "colocacao")[['player']].count().query('player != 1')

,player
colocacao,
0,2
6,2
22,2
44,3


- Concluído que só houveram registros equivocados para as posições 6, 22 e 44;
____________________

Colocações não ocupadas:

In [316]:
colocacoes_partida_4 = set(list(partida_4.colocacao))
colocacoes_possiveis = set(list(range(1, 45)))
colocacoes_faltantes_4 = list(colocacoes_possiveis - colocacoes_partida_4)
print(colocacoes_faltantes_4)
colocacoes_faltantes_4.append(6)
colocacoes_faltantes_4.append(22)
colocacoes_faltantes_4

[37, 39, 41, 43, 20, 31]


[37, 39, 41, 43, 20, 31, 6, 22]

- Colocações a serem preenchidas com tratamento;
____________

Players cujos registros serão tratados nessa partida:

In [318]:
players_a_serem_tratados_partida_4 = partida_4.query('(colocacao == 44 & kills == 0) | colocacao == 0 | colocacao == 6 | colocacao == 22')
players_a_serem_tratados_partida_4

,player,input,partida,kills,colocacao
67,EUCHAVS,controle,4,0,44
59,RIVOTRIL,mouse,4,0,44
251,SAVIOLI,controle,4,1,22
123,FELPIIS32,mouse,4,3,22
235,IGOTEE,mouse,4,1,6
299,XGUSS_,mouse,4,3,6
83,MAUMAU,controle,4,0,0
179,CLARA FIERCE,controle,4,0,0


### Partida 5

In [319]:
partida_5 = final.query('partida == 5').sort_values(by='colocacao', ascending = False)
partida_5.head()

,player,input,partida,kills,colocacao
60,RIVOTRIL,mouse,5,0,44
76,VOORTZ,mouse,5,0,44
292,GARCIA,mouse,5,2,42
100,DETONAMACHO,controle,5,1,40
228,PAPAGUEJO,controle,5,0,40


Players na 44 colocação:

In [320]:
partida_5.query('colocacao == 44')

,player,input,partida,kills,colocacao
60,RIVOTRIL,mouse,5,0,44
76,VOORTZ,mouse,5,0,44


- Não é possível distinguir qual players realmente ficou em 44 lugar;
____________________

Colocações '0':

In [321]:
partida_5.query('colocacao == 0')

,player,input,partida,kills,colocacao
260,SROGATA,controle,5,0,0
236,IGOTEE,mouse,5,0,0


- Os players SROGATA e IGOTEE deverão ser reposicionados nessa partida;
___

Colocações com mais de um player:

In [322]:
partida_5.groupby(by = "colocacao")[['player']].count().query('player != 1')

,player
colocacao,
0,2
3,2
40,2
44,2


- Concluído que só houveram registros equivocados para as posições 3, 40 e 44;
____________________

Colocações não ocupadas:

In [325]:
colocacoes_partida_5 = set(list(partida_5.colocacao))
colocacoes_possiveis = set(list(range(1, 45)))
colocacoes_faltantes_5 = list(colocacoes_possiveis - colocacoes_partida_5)
print(colocacoes_faltantes_5)
colocacoes_faltantes_5.append(3)
colocacoes_faltantes_5.append(40)
colocacoes_faltantes_5.append(44)
colocacoes_faltantes_5

[32, 6, 41, 43, 19]


[32, 6, 41, 43, 19, 3, 40, 44]

- Colocações a serem preenchidas com tratamento;
____________

Players cujos registros serão tratados nessa partida:

In [327]:
players_a_serem_tratados_partida_5 = partida_5.query('(colocacao == 44 & kills == 0) | colocacao == 0 | colocacao == 3 | colocacao == 40 | colocacao == 44')
players_a_serem_tratados_partida_5

,player,input,partida,kills,colocacao
60,RIVOTRIL,mouse,5,0,44
76,VOORTZ,mouse,5,0,44
100,DETONAMACHO,controle,5,1,40
228,PAPAGUEJO,controle,5,0,40
252,SAVIOLI,controle,5,4,3
132,PRUKKZ,controle,5,10,3
260,SROGATA,controle,5,0,0
236,IGOTEE,mouse,5,0,0


### Partida 6

In [328]:
partida_6 = final.query('partida == 6').sort_values(by='colocacao', ascending = False)
partida_6.head()

,player,input,partida,kills,colocacao
165,DREYK,controle,6,0,44
157,NOTGUSTT,mouse,6,0,44
61,RIVOTRIL,mouse,6,0,44
141,ILUDIDO_FPS,controle,6,1,43
21,iErickziN,controle,6,1,41


Players na 44 colocação:

In [329]:
partida_6.query('colocacao == 44')

,player,input,partida,kills,colocacao
165,DREYK,controle,6,0,44
157,NOTGUSTT,mouse,6,0,44
61,RIVOTRIL,mouse,6,0,44


- Não é possível distinguir qual players realmente ficou em 44 lugar;
____________________

Colocações '0':

In [330]:
partida_6.query('colocacao == 0')

,player,input,partida,kills,colocacao
237,IGOTEE,mouse,6,0,0


- O player IGOTEE deverá ser reposicionado nessa partida;
___

Colocações com mais de um player:

In [331]:
partida_6.groupby(by = "colocacao")[['player']].count().query('player != 1')

,player
colocacao,
19,2
44,3


- Concluído que só houveram registros equivocados para as posições 19 e 44;
____________________

Colocações não ocupadas:

In [332]:
colocacoes_partida_6 = set(list(partida_6.colocacao))
colocacoes_possiveis = set(list(range(1, 45)))
colocacoes_faltantes_6 = list(colocacoes_possiveis - colocacoes_partida_6)
print(colocacoes_faltantes_6)
colocacoes_faltantes_6.append(19)
colocacoes_faltantes_6.append(44)
colocacoes_faltantes_6

[40, 18, 3, 42]


[40, 18, 3, 42, 19, 44]

- Colocações a serem preenchidas com tratamento;
____________

Players cujos registros serão tratados nessa partida:

In [333]:
players_a_serem_tratados_partida_6 = partida_6.query('(colocacao == 44 & kills == 0) | colocacao == 0 | colocacao == 19 | colocacao == 44')
players_a_serem_tratados_partida_6

,player,input,partida,kills,colocacao
165,DREYK,controle,6,0,44
157,NOTGUSTT,mouse,6,0,44
61,RIVOTRIL,mouse,6,0,44
133,PRUKKZ,controle,6,2,19
277,THACKOUTZ,mouse,6,5,19
237,IGOTEE,mouse,6,0,0


### Partida 7

In [342]:
partida_7 = final.query('partida == 7').sort_values(by='colocacao', ascending = False)
partida_7.head()

,player,input,partida,kills,colocacao
350,RAFILD,controle,7,2,44
62,RIVOTRIL,mouse,7,0,44
158,NOTGUSTT,mouse,7,0,44
126,FELPIIS32,mouse,7,0,44
102,DETONAMACHO,controle,7,0,44


Players na 44 colocação:

In [343]:
partida_7.query('colocacao == 44')

,player,input,partida,kills,colocacao
350,RAFILD,controle,7,2,44
62,RIVOTRIL,mouse,7,0,44
158,NOTGUSTT,mouse,7,0,44
126,FELPIIS32,mouse,7,0,44
102,DETONAMACHO,controle,7,0,44
70,EUCHAVS,controle,7,0,44
94,KID AIMBOT,controle,7,0,44
246,FOTTLE,controle,7,0,44
38,SURILOCAO,mouse,7,0,44
326,FODOVSKI,mouse,7,0,44


- O registro do player RAFILD é considerado o legítimo e os demais separados para tratamento;
____________________

Colocações '0':

In [344]:
partida_7.query('colocacao == 0')

,player,input,partida,kills,colocacao
238,IGOTEE,mouse,7,0,0


- O player IGOTEE deverá ser reposicionado nessa partida;
___

Colocações com mais de um player:

In [337]:
partida_7.groupby(by = "colocacao")[['player']].count().query('player != 1')

,player
colocacao,
44,10


- Concluído que só houveram registros equivocados para a posição 44;
____________________

Colocações não ocupadas:

In [345]:
colocacoes_partida_7 = set(list(partida_7.colocacao))
colocacoes_possiveis = set(list(range(1, 45)))
colocacoes_faltantes_7 = list(colocacoes_possiveis - colocacoes_partida_7)
print(colocacoes_faltantes_7)

[34, 3, 36, 39, 40, 42, 43, 18, 25, 30]


- Colocações a serem preenchidas com tratamento;
____________

Players cujos registros serão tratados nessa partida:

In [341]:
players_a_serem_tratados_partida_7 = partida_7.query('(colocacao == 44 & kills == 0) | colocacao == 0')
players_a_serem_tratados_partida_7

,player,input,partida,kills,colocacao
62,RIVOTRIL,mouse,7,0,44
158,NOTGUSTT,mouse,7,0,44
126,FELPIIS32,mouse,7,0,44
102,DETONAMACHO,controle,7,0,44
70,EUCHAVS,controle,7,0,44
94,KID AIMBOT,controle,7,0,44
246,FOTTLE,controle,7,0,44
38,SURILOCAO,mouse,7,0,44
326,FODOVSKI,mouse,7,0,44
238,IGOTEE,mouse,7,0,0


### Partida 8

In [346]:
partida_8 = final.query('partida == 8').sort_values(by='colocacao', ascending = False)
partida_8.head()

,player,input,partida,kills,colocacao
135,PRUKKZ,controle,8,5,44
327,FODOVSKI,mouse,8,0,44
63,RIVOTRIL,mouse,8,0,44
79,VOORTZ,mouse,8,0,44
95,KID AIMBOT,controle,8,0,44


Players na 44 colocação:

In [347]:
partida_8.query('colocacao == 44')

,player,input,partida,kills,colocacao
135,PRUKKZ,controle,8,5,44
327,FODOVSKI,mouse,8,0,44
63,RIVOTRIL,mouse,8,0,44
79,VOORTZ,mouse,8,0,44
95,KID AIMBOT,controle,8,0,44
103,DETONAMACHO,controle,8,0,44
167,DREYK,controle,8,5,44


- Não é possível identificar o registro legítimo da 44 colocação.
____________________

Colocações '0':

In [348]:
partida_8.query('colocacao == 0')

,player,input,partida,kills,colocacao
199,BALTAFPS,mouse,8,0,0
119,FINEKSS,mouse,8,0,0
255,SAVIOLI,controle,8,0,0
239,IGOTEE,mouse,8,0,0
351,RAFILD,controle,8,0,0


- Tais players deverão ser reposicionados na partida;
___

Colocações com mais de um player:

In [350]:
partida_8.groupby(by = "colocacao")[['player']].count().query('player != 1')

,player
colocacao,
0,5
44,7


- Concluído que só houveram registros equivocados para a posição 44;
____________________

Colocações não ocupadas:

In [353]:
colocacoes_partida_8 = set(list(partida_8.colocacao))
colocacoes_possiveis = set(list(range(1, 45)))
colocacoes_faltantes_8 = list(colocacoes_possiveis - colocacoes_partida_8)
print(colocacoes_faltantes_8)
colocacoes_faltantes_8.append(44)
colocacoes_faltantes_8

[34, 35, 36, 39, 41, 42, 43, 19, 25, 26, 30]


[34, 35, 36, 39, 41, 42, 43, 19, 25, 26, 30, 44]

- Colocações a serem preenchidas com tratamento;
____________

Players cujos registros serão tratados nessa partida:

In [355]:
players_a_serem_tratados_partida_8 = partida_8.query('(colocacao == 44) | colocacao == 0')
players_a_serem_tratados_partida_8

,player,input,partida,kills,colocacao
135,PRUKKZ,controle,8,5,44
327,FODOVSKI,mouse,8,0,44
63,RIVOTRIL,mouse,8,0,44
79,VOORTZ,mouse,8,0,44
95,KID AIMBOT,controle,8,0,44
103,DETONAMACHO,controle,8,0,44
167,DREYK,controle,8,5,44
199,BALTAFPS,mouse,8,0,0
119,FINEKSS,mouse,8,0,0
255,SAVIOLI,controle,8,0,0
